In [61]:

import napari
import skimage as sk
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from copy import copy
from scipy.signal import find_peaks


In [3]:
im = sk.io.imread('./domes/Dome103_processed.tif')
im = im[1:, :,:] # remove the first image due to its blurriness

In [4]:
# fig, ax = plt.subplot_mosaic([
#     ['1', '2']
# ], figsize=(30,20))


mask = np.empty((0,im.shape[1], im.shape[2])) # empty np array that has 0 Z , 2048 Y , 2048 X
c = 1
for i in im: # Iterate over each YX plane of the image
    i = i.astype(np.uint8)
    gaussian = gaussian_filter(i, sigma=1)
    median = sk.filters.rank.median(gaussian, sk.morphology.star(10))
    # median = median.astype(np.uint8)
    th3 = cv.adaptiveThreshold(median, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 55, -1) # Needs uint8 to make the asaptive threshold mask
    #ax['1'].imshow(median)
    #ax['2'].imshow(th3)
    mask = np.append(mask, np.array(th3))
    # break
    print(c, end="\r")
    c+=1
    


In [46]:
# good nice mask

kernel = np.ones((4,4), np.uint8)
mask_2 = np.empty((0,im.shape[1], im.shape[2])) # empty np array that has 0 Z , 2048 Y , 2048 X

c=1
for i in im:
    img = i.astype(np.uint8)
    gaussian_img = gaussian_filter(img, sigma=1)
    median_img = sk.filters.rank.median(gaussian_img, sk.morphology.star(12))
    # median = median.astype(np.uint8)
    th3_img = cv.adaptiveThreshold(median_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 55, -1) # Needs uint8 to make the asaptive threshold mask
    th3_img = cv.morphologyEx(th3_img, cv.MORPH_OPEN, kernel=kernel)
    th3_img = cv.morphologyEx(th3_img, cv.MORPH_CLOSE, kernel=kernel)
    mask_2 = np.append(mask_2, np.array(th3_img))
    print(c, end="\r")
    c+=1
    

In [88]:
# Try to get rid of the borders
mask_3 = copy(mask_2)
n = -209
c=0
for i in range(len(mask_3)):
    # if c == n:
    # plt.imshow(i)
    peaks, _ = find_peaks(np.sum(mask_3[i], axis=0), height=mask_3[i].shape[1]*255/1.5)
    #plt.plot(np.sum(i, axis=0))
    #plt.plot(peaks,np.sum(i, axis=0)[peaks], "ro")
    for p in peaks:

        mask_3[i][:, p-10: p+14] = 0
    
    
    #break
    c+=1


In [89]:
mask_2 = np.reshape(mask_2, (im.shape[0], im.shape[1], im.shape[2]))
viewer = napari.Viewer()
viewer.add_image(mask_3)

<Image layer 'mask_3' at 0x220f7555390>

In [5]:
skeleton = np.empty((0,im.shape[1], im.shape[2])) # empty np array that has 0 Z , 2048 Y , 2048 X
mask = np.reshape(mask, (im.shape[0], im.shape[1], im.shape[2]))
for i in mask:
    i = sk.morphology.skeletonize(i)
    skeleton = np.append(skeleton, np.array(i))
skeleton = np.reshape(skeleton, (im.shape[0], im.shape[1], im.shape[2]))

In [43]:
thres_line = 200
skeleton_cut = copy(skeleton)
for i in range(skeleton_cut.shape[0]):
    sums = np.sum(skeleton_cut[i], axis=0)
    idx = np.where(sums > thres_line)
    idx = int(np.mean(idx))
    print(idx)
    idx = [idx+i for i in range(-3, 4)]
    for j in idx:
        skeleton_cut[i, :, j] = 0
    

242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
116
115
114
112
111
110
109
108
107
107
106
104
103
102
101
100
99
99
98
96
95
94
93
92
441
438
437
435
434
317
431
430
429
313
312
311
310
309
308
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
284
398
397
281
395
510
393
508
507
506
505
504
503
502
501
500
499
498
497
496
495
494
493
492
375
490
373
372
371
486
485
484
367
482
365
480
479
362
361
360
705
704
703
702
701
700
699
698
697
696
695
694
693
692
691
690
689
688
686
685
685
684
683


In [42]:
viewer = napari.Viewer()
viewer.add_image(skeleton_cut)
viewer.add_image(skeleton)
viewer.add_image(mask, scale=np.array([1,1,1]))

<Image layer 'mask' at 0x220ea1ea990>